In [4]:
# for running on google colab
# !pip install keras
# !pip install tensorflow

In [5]:
# adapted from https://github.com/Conferences2023/TPDL/blob/main/Citation%20Analysis/LSTM.py
'''code of Usman, M., & Balke, W.-T. (2023). On retraction cascade? Citation intention analysis as a quality control mechanism in digital libraries. In O. Alonso, H. Cousijn, G. Silvello, M. Marrero, C. Teixeira Lopes, & S. Marchesin (Eds.), Linking Theory and Practice of Digital Libraries (pp. 117–131). Springer Nature Switzerland. https://doi.org/10.1007/978-3-031-43849-3_11'''

'code of Usman, M., & Balke, W.-T. (2023). On retraction cascade? Citation intention analysis as a quality control mechanism in digital libraries. In O. Alonso, H. Cousijn, G. Silvello, M. Marrero, C. Teixeira Lopes, & S. Marchesin (Eds.), Linking Theory and Practice of Digital Libraries (pp. 117–131). Springer Nature Switzerland. https://doi.org/10.1007/978-3-031-43849-3_11'

In [6]:
import pandas as pd
import numpy as np
import openpyxl
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd

In [8]:
# Load data from CSV file
train_data = pd.read_csv('ML_input_data/train_set.csv')
test_data = pd.read_csv('ML_input_data/test_set.csv')

In [9]:
# Encode labels
le = LabelEncoder()
train_labels = le.fit_transform(train_data['Citation context level annotation'])
test_labels = le.transform(test_data['Citation context level annotation'])

# Tokenize citation context text
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data['Citation context (sent)'])
train_sequences = tokenizer.texts_to_sequences(train_data['Citation context (sent)'])
test_sequences = tokenizer.texts_to_sequences(test_data['Citation context (sent)'])

# Pad sequences to same length
max_len = 100
train_sequences = pad_sequences(train_sequences, maxlen=max_len)
test_sequences = pad_sequences(test_sequences, maxlen=max_len)

# Define neural network architecture
model = Sequential()
model.add(Embedding(5000, 128, input_length=max_len))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
# model.fit(train_sequences, train_labels, epochs=30, batch_size=32, validation_data=(test_sequences, test_labels))
model.fit(train_sequences, train_labels, epochs=30, batch_size=32)

# Make predictions on test set
y_pred = model.predict(test_sequences)
y_pred = (y_pred > 0.5)

# Print classification report
print(classification_report(test_labels, y_pred, target_names=['N', 'Y']))

# Print confusion matrix
print(confusion_matrix(test_labels, y_pred))

Epoch 1/30


/opt/anaconda3/envs/tensorflow/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.5448 - loss: 0.6897
Epoch 2/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.6747 - loss: 0.6489
Epoch 3/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.6275 - loss: 0.6240
Epoch 4/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.6823 - loss: 0.5549
Epoch 5/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.8188 - loss: 0.4052
Epoch 6/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9367 - loss: 0.3109
Epoch 7/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.9645 - loss: 0.1813
Epoch 8/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9786 - loss: 0.1080
Epoch 9/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.9740 - loss: 0.0914
Epoch 10/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.9853 - loss: 0.0681
Epoch 11/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.9860 - loss: 0.0435
Epoch 12/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9959 - loss: 0.0264
Epoch 13/30


In [12]:
# create predictions for all instances
full_data = pd.read_csv('ML_input_data/data_all.csv')
full_sequences = tokenizer.texts_to_sequences(full_data['Citation context (sent)'])
full_sequences = pad_sequences(full_sequences, maxlen=max_len)
full_predictions = model.predict(full_sequences)
full_predictions = (full_predictions > 0.5)
full_data['Predicted label'] = full_predictions
# copy this to the input_data if you wanted to verify the entire workflow
# however, since the output of LSTM is non-deterministic, you might get slightly different results than reported in the paper
full_data.to_csv('ML_output_data/LSTM_prediction_203.csv', index=False)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
